# Testing Notebook 02:

In [18]:
from typing import List
import pandas as pd 
import numpy as np

from statdepth.depth import banddepth
from statdepth.depth.depth import _univariate_band_depth, _handle_depth_errors
from statdepth.depth._containment import _select_containment

import plotly.express as px

In [4]:
!where python

/Users/julian/opt/miniconda3/bin/python
/usr/bin/python


Generate a toy DataFrame for testing

In [7]:
df = pd.DataFrame(np.random.randint(0,101,size=(137, 50)))
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,85,55,64,82,15,21,17,96,20,59,...,66,84,22,15,30,48,36,5,81,77
1,83,13,16,7,85,37,92,25,48,31,...,66,61,55,65,93,5,23,88,8,79
2,96,37,63,100,83,88,99,19,14,35,...,51,43,97,79,8,54,79,41,96,42
3,83,47,100,84,54,62,82,53,57,14,...,36,8,17,27,93,79,51,16,2,100
4,75,32,88,0,36,78,11,42,92,59,...,98,22,28,41,68,46,3,93,85,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,94,68,21,33,29,49,89,75,32,65,...,99,95,85,90,28,40,73,8,10,64
133,31,84,72,54,69,85,81,67,12,83,...,1,41,61,96,87,61,87,81,6,17
134,85,37,51,99,4,50,20,32,67,6,...,79,33,23,34,31,98,61,42,43,88
135,58,63,59,7,88,92,77,94,17,4,...,32,23,67,69,98,98,54,52,94,59


In [9]:
import statdepth as s

In [10]:
%%timeit
s.depth.banddepth([df], J=2)

38.7 s ± 1.07 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
s.depth.samplebanddepth([df], K=3, J=2)

11.6 s ± 446 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
d = s.depth.banddepth([df], J=2, relax=True)
d

0     0.330594
1     0.335218
2     0.342661
3     0.303155
4     0.297524
5     0.333478
6     0.315174
7     0.330594
8     0.337608
9     0.341576
10    0.334253
11    0.327442
12    0.310699
13    0.330064
14    0.324177
15    0.322818
16    0.318272
17    0.349763
18    0.325643
19    0.311217
20    0.334175
21    0.318647
22    0.328509
23    0.324523
24    0.355334
25    0.330797
26    0.350669
27    0.339401
28    0.347660
29    0.339574
30    0.327377
31    0.349298
32    0.336416
33    0.335516
34    0.316330
35    0.331089
36    0.343566
37    0.324546
38    0.306176
39    0.327204
40    0.317593
41    0.344394
42    0.310389
43    0.327341
44    0.338734
45    0.340217
46    0.315960
47    0.327669
48    0.319970
49    0.328485
dtype: float64

In [16]:
sd = s.depth.samplebanddepth([df], K=3, J=2, relax=True)
sd

0     0.302331
1     0.307908
2     0.311978
3     0.278406
4     0.271798
5     0.306895
6     0.286892
7     0.303068
8     0.310498
9     0.312606
10    0.306777
11    0.299562
12    0.286445
13    0.301301
14    0.295951
15    0.297320
16    0.292592
17    0.321471
18    0.299510
19    0.283612
20    0.307144
21    0.293122
22    0.302300
23    0.298331
24    0.325868
25    0.303435
26    0.318850
27    0.311409
28    0.321942
29    0.313284
30    0.300417
31    0.318864
32    0.307395
33    0.309909
34    0.290519
35    0.303100
36    0.315748
37    0.298730
38    0.280776
39    0.300565
40    0.290802
41    0.318002
42    0.283976
43    0.301962
44    0.310287
45    0.311977
46    0.291126
47    0.300136
48    0.293871
49    0.298904
dtype: float64

In [17]:
d - sd

0     0.028264
1     0.027311
2     0.030682
3     0.024749
4     0.025727
5     0.026583
6     0.028281
7     0.027527
8     0.027110
9     0.028970
10    0.027476
11    0.027880
12    0.024254
13    0.028763
14    0.028226
15    0.025498
16    0.025680
17    0.028292
18    0.026133
19    0.027605
20    0.027031
21    0.025526
22    0.026209
23    0.026191
24    0.029466
25    0.027362
26    0.031818
27    0.027992
28    0.025718
29    0.026290
30    0.026959
31    0.030435
32    0.029021
33    0.025607
34    0.025810
35    0.027989
36    0.027818
37    0.025817
38    0.025400
39    0.026639
40    0.026791
41    0.026392
42    0.026413
43    0.025379
44    0.028447
45    0.028240
46    0.024834
47    0.027533
48    0.026099
49    0.029581
dtype: float64

In [21]:
l = [3, 5, 4, 6.5, 8, 5, 4.5, 2, 4] * 4
px.line(l)

In [24]:
df = pd.DataFrame()
for k in range(10):
    df[k] = [i * np.random.rand() for i in l]
    
px.line(df)

In [39]:
import statdepth

s = statdepth.depth.banddepth([df], J=2, relax=True)
s

0    0.288889
1    0.303704
2    0.269136
3    0.251852
4    0.279012
5    0.256790
6    0.260494
7    0.256790
8    0.254321
9    0.245679
dtype: float64

In [42]:
sd = statdepth.depth.samplebanddepth([df], K=2, J=2, relax=True)
sd

0    0.223611
1    0.249537
2    0.219444
3    0.202778
4    0.222222
5    0.208333
6    0.209722
7    0.198148
8    0.201852
9    0.207407
dtype: float64

In [41]:
s-sd

0    5.551115e-17
1    0.000000e+00
2    5.551115e-17
3    5.551115e-17
4    0.000000e+00
5    0.000000e+00
6    0.000000e+00
7    5.551115e-17
8    0.000000e+00
9    2.775558e-17
dtype: float64